## Plotting, frequency analysis

Sections in this notebook:

1. Pick a station
2. Plot
3. What's next? 

In [6]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 8, 6

import obspy
from obspy import UTCDateTime
from obspy.imaging import spectrogram

import ipywidgets as widgets
from ipywidgets import HBox, VBox, interact, Layout
style = {'description_width': 'initial'}
layout=Layout(width='30%', height='40px')

***
## 1. Pick a station
***

In [7]:
# Assign station codes from selected file

dir = './DataFiles/Level2/'
sta_list = []
for file in os.listdir(dir):
    if file.endswith('regional_strain_and_corrections.txt'):
        sta_list.append(file[0:24])
        
# Set initial values   
file = sta_list[0]
network = file[0:2]
scode = file[3:7]
loc = file[8:10]
cha = file[11:13]

print('Pick the file you would like to analyze:')
sta_select = widgets.Dropdown(
            options=sta_list,
            value=sta_list[0],
            description='Station file:',
            )
display(sta_select)

# Change the station and network as the dropdown is changed
def the_ccodes(siteval):
    global scode, network, loc, cha, file
    file = siteval
    network = siteval[0:2]
    scode = siteval[3:7]
    loc = siteval[8:10]
    cha = siteval[11:13]
def on_cselect(change):
    the_ccodes(change.new)
sta_select.observe(on_cselect,names='value')

# Load the strains and corrections to a dataframe

# Make a dataframe with the file and assign start and end dates
fbutton = widgets.Button(description="Load files to a dataframe", button_style='danger',layout=layout)
foutput = widgets.Output()

# Initial start and end times (arbitrarily chosen)
start = UTCDateTime('2000-01-01 00:00:00.000')
end = UTCDateTime('2000-01-02 00:00:00.000')

def on_fbutton_clicked(b):
    with foutput:
        foutput.clear_output()
        global df, start, end
        df = pd.DataFrame([])
        for files in os.listdir(dir):
            if files.startswith(file):
                # Print file comments
                with open(dir+files,'r') as f:
                    for ln in f:
                        if ln.startswith('#'):
                            print('File comment: '+ ln[1:])
                f.close()
                adf = pd.read_csv(dir+files,sep='\t',index_col=0,header=0,comment='#')
                df = pd.concat([df,adf],axis='columns')
        print('Wait for the dataframe column headers to print.')

        ind = []
        for i in range(0,len(df)):
            ind.append(UTCDateTime(df.index[i]))
        df.index = ind
        start = df.index[0]
        end = df.index[-1]
        print(df.columns)
fbutton.on_click(on_fbutton_clicked)


display(fbutton, foutput)

Pick the file you would like to analyze:


Dropdown(description='Station file:', options=('PB.B073.T0.LS.2012-01-21', 'PB.B916.T0.RS.2009-02-06'), value=…

Button(button_style='danger', description='Load files to a dataframe', layout=Layout(height='40px', width='30%…

Output()

***
## 2. Plot
***

In [8]:
# Interactive plotting
        
# Set initial values
correct = {'Pressure':1, 'Linear':2, 'Tides':3}
A = widgets.SelectMultiple(
    options=correct,
    description='Corrections to apply:',
    disabled=False, style=style
    )

plot = {'ch0':0, 'ch1':1, 'ch2':2, 'ch3':3, 'Areal':'EA','Differential Shear':'ED','Engineering Shear':'ES'}
B = widgets.SelectMultiple(
    options=plot,
    description='Plot:',
    disabled=False, style=style
    )
plot_corr = {'Pressure Correction':1,'Modelled Tides':2,'Linear Trend':3}
C = widgets.SelectMultiple(
    options=plot_corr,
    description='Include Correction:',
    disabled=False, style=style
    )
display()
pbutton = widgets.Button(description="Plot", button_style='danger')
poutput = widgets.Output()

def on_pbutton_clicking(but):
    with poutput:
        poutput.clear_output()
        print('Working on it...')
        plt.close()
        cor_ch = {'0':None,'1':None,'2':None,'3':None}
        for ch in cor_ch:
            if (1 in A.value) == True:
                p = 1
            else:
                p = 0
            if (2 in A.value) == True:
                l = 1
            else:
                l = 0
            if (3 in A.value) == True: 
                t = 1
            else: t = 0
            # Corrected gauge data
            cor_ch[ch] = df['ch'+ch+' [ms]'] - df['baro_ch'+ch] * p - df['trend_ch'+ch] * l - df['tide_ch'+ch] * t
        # if areal and/or shears are selected, apply corrections
        cor_reg = {'EA':None,'ED':None,'ES':None}
        for reg in cor_reg:
            # corrected regional strains
            cor_reg[reg] = df['gauge'+reg] - df['baro'+reg] * p - df['trend'+reg] * l - df['tide'+reg] * t
        # Nice time for plotting
        xtime = (df.index - df.index[0])/60/60/24
        # if gauges are selected, plot gauge strain
        if (0 in B.value) == True: plt.plot(xtime,cor_ch['0'],label='ch0')
        if (1 in B.value) == True: plt.plot(xtime,cor_ch['1'],label='ch1')
        if (2 in B.value) == True: plt.plot(xtime,cor_ch['2'],label='ch2')
        if (3 in B.value) == True: plt.plot(xtime,cor_ch['3'],label='ch3')
        # if regional strains are selected, plot regional strains
        if ('EA' in B.value) == True: plt.plot(xtime,cor_reg['EA'],label='Areal')
        if ('ED' in B.value) == True: plt.plot(xtime,cor_reg['ED'],label='Differential Shear')
        if ('ES' in B.value) == True: plt.plot(xtime,cor_reg['ES'],label='Engineering Shear')
        # if corrections are selected, plot the corrections
        correction = []
        for i in [1,2,3]:
            if (1 in C.value) == True:
                correction.append('baro')
            else:
                correction.append(0)
            if (2 in C.value) == True:
                correction.append('tide')
            else:
                correction.append(0)
            if (3 in C.value) == True:
                correction.append('trend')
            else:
                correction.append(0)
            if correction[i] != 0:
                if 0 in B.value: plt.plot(xtime,df[correction[i]+'_ch0'],label='ch0 '+correction[i]) 
                if 1 in B.value: plt.plot(xtime,df[correction[i]+'_ch1'],label='ch1 '+correction[i]) 
                if 2 in B.value: plt.plot(xtime,df[correction[i]+'_ch2'],label='ch2 '+correction[i]) 
                if 3 in B.value: plt.plot(xtime,df[correction[i]+'_ch3'],label='ch3 '+correction[i]) 
                if 'EA' in B.value: plt.plot(xtime,df[correction[i]+'EA'],label='EA '+correction[i])
                if 'ED' in B.value: plt.plot(xtime,df[correction[i]+'ED'],label='ED '+correction[i])
                if 'ES' in B.value: plt.plot(xtime,df[correction[i]+'ES'],label='ES '+correction[i])
        plt.legend()
        plt.title(scode+' strain')
        plt.xlabel('Days from '+str(start)[0:10]+' '+str(start)[11:19])
        plt.ylabel('Microstrain')
pbutton.on_click(on_pbutton_clicking)

display(VBox([HBox([A,B,C]),pbutton, poutput]))

***
## 3. What next?
***

It's up to you! You could do a frequency analysis, examine the residuals, take a look at pore pressure sensor info for the station... there is quite a lot of science to be done. All of the data files and notebooks changes you have made so far are saved on your machine in your `/bsm_tutorial/notebooks/` directory.

Matplotlib has power spectral density and spectrogram utilities that you may find interesting. Here is some example code that could be modified:

`samp_freq =(len(df)-1)/(end-start)
NQ = samp_freq*2
plt.close()` <br>
choose <br>
`plt.psd(df['ch0 [ms]'].values,Fs=samp_freq,NFFT=2048)` <br>
OR <br>
`plt.specgram(df['ch0 [ms]'].values,Fs=samp_freq,NFFT=1024)`


In [9]:
# Make a spectrogram
samp_freq =(len(df)-1)/(end-start)
NQ = samp_freq*2
plt.close()
plt.psd(df['ch0 [ms]'].values,Fs=samp_freq,NFFT=2048)
#plt.specgram(df['ch0 [ms]'].values,Fs=samp_freq,NFFT=2048)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  5.10165014e+00,   2.57294826e+00,   1.44412078e-03, ...,
          1.33160759e-08,   1.45781151e-08,   9.08806204e-09]),
 array([  0.00000000e+00,   4.88281250e-04,   9.76562500e-04, ...,
          4.99023438e-01,   4.99511719e-01,   5.00000000e-01]))